In [ ]:
import os
Root = "/content/drive/MyDrive/ml proj/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [ ]:
ls

Actor_01/  Actor_06/  Actor_11/  Actor_16/  Actor_21/
Actor_02/  Actor_07/  Actor_12/  Actor_17/  Actor_22/
Actor_03/  Actor_08/  Actor_13/  Actor_18/  Actor_23/
Actor_04/  Actor_09/  Actor_14/  Actor_19/  Actor_24/
Actor_05/  Actor_10/  Actor_15/  Actor_20/  modelForPrediction1.sav


In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/ml proj/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
x_train

array([[-4.95599945e+02,  2.82138557e+01, -5.80377722e+00, ...,
         3.14757839e-04,  2.04955722e-04,  1.31822366e-04],
       [-6.42915588e+02,  7.66249466e+01,  2.75691414e+00, ...,
         4.64339200e-06,  2.52479435e-06,  1.17451077e-06],
       [-5.55030457e+02,  2.36032677e+01, -2.53534470e+01, ...,
         7.57200105e-05,  4.78267320e-05,  2.09626596e-05],
       ...,
       [-5.78326233e+02,  2.42516861e+01, -4.97860098e+00, ...,
         1.09455046e-04,  7.40213218e-05,  3.72870491e-05],
       [-4.71528381e+02,  3.68739777e+01, -6.11022472e+00, ...,
         4.36598057e-04,  2.39399931e-04,  1.13480164e-04],
       [-4.80286407e+02,  2.78660526e+01, -1.06364164e+01, ...,
         1.85660459e-03,  1.39153481e-03,  8.21309222e-04]])

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['disgust', 'happy', 'calm', 'happy', 'happy', 'disgust', 'happy',
       'happy', 'fearful', 'happy', 'happy', 'calm', 'calm', 'calm',
       'calm', 'fearful', 'happy', 'calm', 'calm', 'fearful', 'calm',
       'calm', 'happy', 'fearful', 'happy', 'fearful', 'disgust',
       'disgust', 'calm', 'happy', 'calm', 'happy', 'disgust', 'happy',
       'calm', 'disgust', 'happy', 'fearful', 'calm', 'fearful', 'happy',
       'happy', 'disgust', 'calm', 'fearful', 'fearful', 'disgust',
       'happy', 'happy', 'happy', 'fearful', 'happy', 'happy', 'calm',
       'calm', 'happy', 'calm', 'fearful', 'fearful', 'calm', 'happy',
       'fearful', 'calm', 'fearful', 'calm', 'calm', 'fearful', 'fearful',
       'happy', 'happy', 'fearful', 'disgust', 'fearful', 'calm', 'happy',
       'happy', 'happy', 'happy', 'calm', 'calm', 'fearful', 'calm',
       'happy', 'fearful', 'happy', 'calm', 'fearful', 'calm', 'happy',
       'calm', 'fearful', 'calm', 'calm', 'calm', 'calm', 'fearful',
      

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 67.71%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.81355932, 0.43243243, 0.72527473, 0.65346535])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,disgust,disgust
1,happy,happy
2,calm,calm
3,disgust,happy
4,happy,happy
5,disgust,disgust
6,happy,happy
7,disgust,happy
8,fearful,fearful
9,happy,happy


In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/ml proj/speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['calm'], dtype='<U7')

In [ ]:
feature

array([[-6.93497009e+02,  5.00643921e+01,  5.71450531e-01,
         1.43299656e+01,  3.33637071e+00, -2.54071975e+00,
        -4.05790949e+00, -1.07119999e+01, -7.29413939e+00,
         1.74018872e+00, -4.19064236e+00,  1.95466173e+00,
        -5.24789381e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803197e+00, -5.68717539e-01,
        -6.47753334e+00, -1.24320555e+00, -2.80542517e+00,
        -5.43635798e+00, -4.46875393e-01, -3.63516593e+00,
        -2.98372602e+00, -5.63902557e-01, -1.65101993e+00,
        -5.55944741e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821292e+00,
        -5.54154456e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577074e+00, -1.62457860e+00, -2.03990698e+00,
        -3.62910867e+00,  6.33751631e-01,  6.48747087e-01,
         6.21731520e-01,  6.34555459e-01,  6.60326064e-01,
         6.60485923e-01,  7.00930357e-01,  7.31556237e-01,
         7.46886790e-01,  7.23434567e-01,  7.16107428e-0